In [4]:
import av
from transformers import AutoImageProcessor, TimesformerForVideoClassification
import cv2
import torch
import numpy as np

# Load the pre-trained image processor and model
processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k600")
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k600")

# Read the MP4 file and extract frames
file_path = r"C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Videos_postprocessed\0013_processed.mp4"
container = av.open(file_path)

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])



def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)

    # Ensure that seg_len is greater than converted_len
    if seg_len <= converted_len:
        # Adjust the clip length if the video is too short
        converted_len = seg_len - 1
        clip_len = converted_len // frame_sample_rate
        if clip_len == 0:
            raise ValueError("Video is too short for the given frame sample rate.")

    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len, endpoint=False)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# Adjust these parameters to match the model's expected input dimensions
clip_length = 112  # Adjusted to match the temporal dimension
frame_sample_rate = 1  # Sample every frame, adjust as needed

indices = sample_frame_indices(clip_len=clip_length, frame_sample_rate=frame_sample_rate, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

# Additional frame processing might be required here

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")

inputs = image_processor(list(video), return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


config.json:   0%|          | 0.00/22.7k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/486M [00:00<?, ?B/s]

RuntimeError: shape '[1, 122, 14, 8, 768]' is invalid for input of size 10536960